In [ ]:
import PIL
from vllm import LLM
from transformers import AutoProcessor

llm = LLM(
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    # trust_remote_code=True,  # Required to load Phi-3.5-vision
    # max_model_len=4096,  # Otherwise, it may not fit in smaller GPUs
    limit_mm_per_prompt={"image": 2},  # The maximum number to accept
    dtype="float16",
    tensor_parallel_size = 4
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")


WARNING 05-16 12:46:24 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-16 12:46:24 [config.py:717] This model supports multiple tasks: {'embed', 'score', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
WARNING 05-16 12:46:24 [arg_utils.py:1536] The model has a long context length (128000). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
INFO 05-16 12:46:24 [config.py:1770] Defaulting to use mp for distributed inference
INFO 05-16 12:46:24 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen2.5-VL-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=128000, download_dir=None, load_format=auto, ten

In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "assets/chatbot.png",
            },
            {
                "type": "image",
                "image": "assets/folder.png",
            },
            {"type": "text", "text": "Describe these two images."},
        ],
    }
]
prompt = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# prompt = '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|><|vision_start|><|image_pad|><|vision_end|>Describe these image.<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
# Load the images using PIL.Image
image1 = PIL.Image.open("assets/chatbot.png")
image2 = PIL.Image.open("assets/folder.png")
outputs = llm.generate({
    "prompt": prompt,
    "multi_modal_data": {
        "image": [image1, image2]
    },
})

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

WARNING 05-16 12:46:14 [config.py:1239] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.
WARNING 05-16 12:46:14 [sampling_params.py:347] temperature 1e-06 is less than 0.01, which may cause numerical errors nan or inf in tensors. We have maxed it out to 0.01.


ValueError: You set or defaulted to '{"image": 1}' in `--limit-mm-per-prompt`, but passed 2 image items in the same prompt.